In [14]:
from posixpath import join
from astropy.io.fits import hdu
import pandas as pd 
from matplotlib import pyplot as plt 
import numpy as np 
from ciao_contrib.runtool import search_csc 

from astropy.io import fits 
from fields import fields 
from tqdm import tqdm


In [ ]:


df = pd.read_csv('../source_list/CV_data_clean_source_list.csv' , delimiter=',' , comment='#' ,)
df = df.replace(' ' , '_' ,regex=True)
df.insert(0  , 'type' , ['CV']*len(df))



start = int(input('Start number : '))
end = int(input('End number : '))

#start = 0 
#end = 3

df_select = df.iloc[start:end].reset_index(drop=True)

ra = df_select['B_RA']
dec = df_select['B_DEC']
name = df_select['A_NAME']
num_obs = [] 
offset = []
for index in tqdm(range(len(df_select))):
#for index in tqdm(range(3)):
    try :
        s = search_csc(
                        radunit="arcsec", 
                        columns=fields, 
                        bands="broad",
                        clobber="yes" ,
                        radius= 3 , 
                        pos= str(ra[index])+','+str(dec[index])  ,
                        outfile='temp.csv')
                    #sys('search_csc outfile=trial.csv radunit=arcsec columns="SOS,SOP,SOV , o.gti_obs m.flux_aper_b" bands=broad clobber=yes radius=1 pos="65.428058,32.907468"')
                    #print(s)
                    
        data = pd.read_csv('temp.csv', delimiter='\t' , comment='#')
        data = data[data['match_type']=='u          ']
        data = data[data['instrument']=='ACIS']
        data.index.name= 'index'
        #f_name_fits = data_fits['A_NAME']
        n = len(data)
        num_obs.append(n)
        if(n>0):
            print('Match found --' , len(data))
            data.to_csv('../data/all_data/cv_old_data_all/'+name[index]+'.csv')
    except:
        #print(e)
        print('Manual ispection needed :' , name[index])
        num_obs.append(-1)
    print(num_obs)

df_updated = df_select.copy()
df_updated.insert(10 , 'num_obs' , num_obs)

df_updated.to_csv('../source_list/cv_old_updated/'+str(start)+'-'+str(end)+'.csv')

In [6]:
df_select

,type,SRC_ID,A_NAME,A_RA,A_DEC,A_DATABASE_TABLE,A_OBSERVATORY,B_NAME,B_RA,B_DEC,XOFFSET_
0,CV,CV0001,[DSH2013]_24___________,204.083083,-29.852333,M83XMM__________,XMM-NEWTON___,2CXO_J133620.0-295106_,204.083487,-29.851710,0.043
1,CV,CV0002,[HPH2013]_100__________,10.654490,41.241180,M31CFCXO________,CHANDRA______,2CXO_J004237.0+411428_,10.654579,41.241130,0.005
2,CV,CV0003,[HPH2013]_101__________,10.655350,41.286130,M31CFCXO________,CHANDRA______,2CXO_J004237.3+411709_,10.655445,41.286063,0.006


In [5]:
num_obs

[3, -1, 33, -1, 0]